In [2]:
import subprocess
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.triggers.interval import IntervalTrigger
import atexit

from flask import Flask, jsonify, request
from flask_cors import CORS, cross_origin
import pandas as pd
import numpy as np

import covid19_model as model

In [3]:
access_count = 0

scheduler = BackgroundScheduler()
scheduler.start()

scheduler.add_job(
    func=model.calculate_forecast,
    trigger='cron',
    hour='3', 
    minute='00')

atexit.register(lambda: scheduler.shutdown())

INFO:apscheduler.scheduler:Scheduler started
INFO:apscheduler.scheduler:Added job "calculate_forecast" to job store "default"


<function __main__.<lambda>()>

In [6]:
app = Flask(__name__)
CORS(app)

@app.route("/katana-ml/api/v1.0/forecast/covid19", methods=['POST'])
def forecast():
    global access_count
    access_count += 1
    
    country = request.json['country']
    
    df = pd.read_csv('data/covid19_forecast_data_' + country + '.csv', parse_dates=True)
    df = df.drop(df.columns[[0]], axis=1)
    
    result = df.to_json(orient='records', date_format='iso')
    return result

@app.route("/katana-ml/api/v1.0/forecast/covid19/count", methods=['POST'])
def g():
    global access_count
    return str(access_count)

# running REST interface port=3000
if __name__ == "__main__":
    app.run(debug=False, host='0.0.0.0', port=3000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://0.0.0.0:3000/ (Press CTRL+C to quit)
INFO:werkzeug:172.17.0.1 - - [03/Apr/2020 11:13:45] "POST /katana-ml/api/v1.0/forecast/covid19 HTTP/1.1" 200 -
INFO:werkzeug:172.17.0.1 - - [03/Apr/2020 11:14:33] "POST /katana-ml/api/v1.0/forecast/covid19/count HTTP/1.1" 200 -
INFO:werkzeug:172.17.0.1 - - [03/Apr/2020 11:14:37] "POST /katana-ml/api/v1.0/forecast/covid19 HTTP/1.1" 200 -
INFO:werkzeug:172.17.0.1 - - [03/Apr/2020 11:14:40] "POST /katana-ml/api/v1.0/forecast/covid19/count HTTP/1.1" 200 -
ERROR:flask.app:Exception on /katana-ml/api/v1.0/forecast/covid19 [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/flask/app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.6/site-packages/flask/app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.6/site-packages/flask_cors/extension.py", line 161, in wrapped_func